In [1]:
import spacy
import os
import numpy as np
import pandas as pd
from google.colab import drive
import requests

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/CAPSTONE/data')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/CAPSTONE/data'

In [ ]:
os.listdir()

In [ ]:
# import csv file, drop irrelevant column
#data = pd.read_csv('Will_1k_set_1.csv')
data = pd.read_csv('combined_data.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,file_name,wiki_commons_id,wiki_data_id,depicts,description,parsed caption,commons_description
0,File:Plac Wielkopolski w Poznaniu - czerwiec 2...,60332930,['Q11819181'],['Wielkopolski Square in Poznań'],"['city square in Poznań, Poland']",Plac Wielkopolski w Poznaniu.\n\n,"{'DateTime': {'value': '2017-06-24 03:53:38', ..."
1,File:2018-10-10 Gymnastics at 2018 Summer Yout...,77382844,['Q93888038'],['Yin Dehang'],['Chinese artistic gymnast'],Horizontal bar qualification of the boys' arti...,"{'DateTime': {'value': '2019-03-16 21:07:55', ..."
2,File:CP 00118destructions rue de Tournai 1914.jpg,84142136,['Q39074508'],['Siege of Lille'],['1914 siege'],Lille (France - Nord department) - Destruction...,"{'DateTime': {'value': '2019-11-19 16:24:13', ..."
3,"File:Waycross, Georgia Historic District (40).jpg",86478066,['Q30622481'],['Waycross Historic District'],"['historic district in Waycross, Georgia']","Waycross Historic District, Waycross, Georgia\...","{'DateTime': {'value': '2020-02-01 01:11:58', ..."
4,File:J31 867 Kraftwerkskaverne.jpg,138279755,['Q963375'],['diesel generator'],['combination of a diesel engine with an elect...,Zwei Dieselgeneratoren für die Gesamtversorgun...,"{'DateTime': {'value': '2023-07-16 20:54:40', ..."


## Original Spacy Model

In [ ]:
# Load the English language model for SpaCy
nlp = spacy.load("en_core_web_sm")

def extract_tags(description):
  """
  This function takes an image description as input and extracts potential tags.

  Args:
      description (str): The text description of the image.

  Returns:
      list: A list of potential tags extracted from the description.
  """
  # Preprocess the text
  description = description.lower()
  description = description.strip() # Remove leading/trailing whitespaces

  # Perform NLP with spaCy
  doc = nlp(description)

  # Extract potential tags
  tags = []
  prev_token = None
  for token in doc:
    if token.pos_ in ["NOUN", "PROPN"]: # Consider nouns and proper nouns
      if prev_token and prev_token.pos_ in ["NOUN", "PROPN"]:
        # Combine the current and previous token for multi-word tags
        tags.append(prev_token.text + " " + token.text)
      else:
        tags.append(token.text)
    prev_token = token

  return tags

In [ ]:

# Extracting a single string, test usage
image_description = "A photo of a Golden Gate Bridge at sunset, with sailboats in the foreground."
tags = extract_tags(image_description)

print("Extracted tags:", tags)

Extracted tags: ['photo', 'golden', 'golden gate', 'gate bridge', 'sunset', 'sailboats', 'foreground']


In [ ]:
# Download larger model (medium), has to be repeated every time Colab session is opened
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## SpaCy update to remove articles from tags

In [ ]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_md")

# Example text from an image description
text = "The photo shows the Eiffel Tower at sunset, surrounded by tourists taking photographs."

# Process the text
doc = nlp(text)

def remove_articles(phrase):
    # List of articles to remove
    articles = ['the', 'a', 'an']
    words = phrase.split()
    # Remove the first word if it is an article
    if words[0].lower() in articles:
        return ' '.join(words[1:])
    return phrase

# Extract noun chunks as potential tags
noun_chunks = [remove_articles(chunk.text) for chunk in doc.noun_chunks]

# Extract named entities as specific tags
named_entities = [remove_articles(entity.text) for entity in doc.ents]

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
tags = list(set(noun_chunks + named_entities))

print("Extracted Tags:", tags)

Extracted Tags: ['Eiffel Tower', 'photographs', 'photo', 'sunset', 'tourists']


## Test - Stemming:

In [ ]:
# download large SpaCy Model for testing purposes
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Incorporate Lemmatization

In [ ]:

# Load the SpaCy model
nlp = spacy.load("en_core_web_lg")

# Example text from an image description
#text = "['Waycross Historic District']"
text = "['Horizontal bar qualification of the boys' artistic gymnastics at the 2018 Summer Youth Olympics in Buenos Aires on 10 October 2018. Depicted: Yin Dehang.']"
# A photo of a Golden Gate Bridge at sunset, with sailboats in the foreground.

# Process the text with SpaCy
doc = nlp(text)

# Function to remove leading articles and lemmatize
def clean_and_lemmatize(phrase, doc):
    articles = ['the', 'a', 'an']
    words = phrase.split()
    # Remove the first word if it is an article
    if words[0].lower() in articles:
        words = words[1:]
    # Lemmatize each word in the phrase
    lemmatized_words = []
    for word in words:
        # Find the token for this word (use the original text for matching)
        tokens = [token for token in doc if token.text.lower() == word.lower()]
        if tokens:
            # Use the lemma of the first matching token
            lemma = tokens[0].lemma_
        else:
            lemma = word  # Fallback to the original word if not found
        lemmatized_words.append(lemma)
    return ' '.join(lemmatized_words)

# Extract noun chunks and clean them
noun_chunks = [clean_and_lemmatize(chunk.text, doc) for chunk in doc.noun_chunks]

# Extract named entities, clean and lemmatize
named_entities = [clean_and_lemmatize(entity.text, doc) for entity in doc.ents]

# Combine both sets of tags, removing duplicates
tags = list(set(noun_chunks + named_entities))

print("Extracted Tags:", tags)

Extracted Tags: ['2018 Summer Youth Olympics', 'Summer Youth Olympics', '10 October 2018', "boys' artistic gymnastic", 'Buenos Aires', '2018', '10 October', 'Yin Dehang', "['Horizontal bar qualification"]


In [ ]:

'''
def clean_and_lemmatize(phrase, doc):
    articles = ['the', 'a', 'an']
    words = phrase.split()
    if words and words[0].lower() in articles:
        words = words[1:]
    lemmatized_words = []
    for word in words:
        tokens = [token for token in doc if token.text.lower() == word.lower()]
        if tokens:
            lemmatized_words.append(tokens[0].lemma_)  # Append lemma to the list
        else:
            lemmatized_words.append(word)  # Append original word if no token found
    return ' '.join(lemmatized_words)
  '''
import spacy
import re

nlp = spacy.load("en_core_web_lg")
def clean_and_lemmatize(phrase, doc):
    articles = ['the', 'a', 'an']
    words = phrase.split()
    # Remove the first word if it is an article
    if words[0].lower() in articles:
        words = words[1:]
    # Lemmatize each word in the phrase
    lemmatized_words = []
    for word in words:
        # Find the token for this word (use the original text for matching)
        tokens = [token for token in doc if token.text.lower() == word.lower()]
        if tokens:
            # Use the lemma of the first matching token
            lemma = tokens[0].lemma_
        else:
            lemma = word  # Fallback to the original word if not found
        lemmatized_words.append(lemma)
    return ' '.join(lemmatized_words)

def process_text(text):
  try:
    #stripped_text = text.strip("['").strip("']")
    #doc = nlp(stripped_text)
    doc = nlp(text)
    url_pattern = r'https?://[^\S\n]*(?:[^\s<>"]|[^\S\n])+'
    text = re.sub(url_pattern, '', text, flags=re.MULTILINE)
    #noun_chunks = [clean_and_lemmatize(chunk.text, doc) for chunk in doc.noun_chunks]
    noun_chunks = [clean_and_lemmatize(chunk.text, doc) for chunk in doc.noun_chunks]
    named_entities = [clean_and_lemmatize(entity.text, doc) for entity in doc.ents]
    tags = list(set(noun_chunks + named_entities))
    filtered_tags = [tag for tag in tags if not (tag.lower().startswith('http') or not tag[0].isalnum())]
    return filtered_tags
  except:
    return 'no tags found'

# Apply the function to the DataFrame
#results = data['parsed caption'].apply(process_text)
#print(results)

In [ ]:
#data['extracted_tags'] = data['parsed caption'].apply(process_text)
data['extracted_tags'] = data['parsed caption'].apply(process_text)

In [ ]:
data.head()

,file_name,wiki_commons_id,wiki_data_id,depicts,description,parsed caption,commons_description,extracted_tags
0,File:Plac Wielkopolski w Poznaniu - czerwiec 2...,60332930,['Q11819181'],['Wielkopolski Square in Poznań'],"['city square in Poznań, Poland']",Plac Wielkopolski w Poznaniu.\n\n,"{'DateTime': {'value': '2017-06-24 03:53:38', ...",[Plac Wielkopolski w Poznaniu]
1,File:2018-10-10 Gymnastics at 2018 Summer Yout...,77382844,['Q93888038'],"['Yin Dehang', '2018 Summer Youth Olympics']",['Chinese artistic gymnast'],Horizontal bar qualification of the boys' arti...,"{'DateTime': {'value': '2019-03-16 21:07:55', ...","[10 October, boys' artistic gymnastic, 2018, Y..."
2,File:CP 00118destructions rue de Tournai 1914.jpg,84142136,['Q39074508'],['Siege of Lille'],['1914 siege'],Lille (France - Nord department) - Destruction...,"{'DateTime': {'value': '2019-11-19 16:24:13', ...","[rue de Tournai, Siege of Lille, October, figh..."
3,"File:Waycross, Georgia Historic District (40).jpg",86478066,['Q30622481'],['Waycross Historic District'],"['historic district in Waycross, Georgia']","Waycross Historic District, Waycross, Georgia\...","{'DateTime': {'value': '2020-02-01 01:11:58', ...","[America](https://en.wikipedia.org, Historic P..."
4,File:J31 867 Kraftwerkskaverne.jpg,138279755,['Q963375'],['diesel generator'],['combination of a diesel engine with an elect...,Zwei Dieselgeneratoren für die Gesamtversorgun...,"{'DateTime': {'value': '2023-07-16 20:54:40', ...","[Motoren sind, der Wand enthalten Stickstoff f..."


In [ ]:
data = data[data['extracted_tags'] != 'no tags found']

In [ ]:
n = len(data)
indices = np.random.permutation(n)
train_size = int(0.8 * n)  # 80% of the total number
val_size = int(0.1 * n)    # 10% of the total number
test_size = n - train_size - val_size  # The remaining 10%, ensuring no rounding issues
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

train_data = data.iloc[train_indices]
val_data = data.iloc[val_indices]
test_data = data.iloc[test_indices]

# Neural Net

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from torch import nn, optim

In [ ]:
class Config:
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    MAX_LEN = 128
    BATCH_SIZE = 8
    LEARNING_RATE = 1e-5
    EPOCHS = 3

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class BertForTagPrediction(nn.Module):
    def __init__(self):
        super(BertForTagPrediction, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.similarity = nn.CosineSimilarity(dim=1)

    def forward(self, input_ids, attention_mask, tag_input_ids, tag_attention_mask):
        # Get embeddings for descriptions
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        description_embeddings = outputs.pooler_output

        # Get embeddings for tags
        tag_outputs = self.bert(input_ids=tag_input_ids, attention_mask=tag_attention_mask)
        tag_embeddings = tag_outputs.pooler_output

        # Calculate similarity
        return self.similarity(description_embeddings, tag_embeddings)

In [ ]:
class TagsDataset(Dataset):
    def __init__(self, descriptions, all_tags):
        self.input_ids = []
        self.attention_masks = []
        self.tag_input_ids = []
        self.tag_attention_masks = []

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        for description, tags in zip(descriptions, all_tags):
            description_enc = tokenizer(description, max_length=Config.MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')
            for tag in tags:
                tag_enc = tokenizer(tag, max_length=Config.MAX_LEN, padding='max_length', truncation=True, return_tensors='pt')

                self.input_ids.append(description_enc['input_ids'].squeeze(0))
                self.attention_masks.append(description_enc['attention_mask'].squeeze(0))
                self.tag_input_ids.append(tag_enc['input_ids'].squeeze(0))
                self.tag_attention_masks.append(tag_enc['attention_mask'].squeeze(0))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'tag_input_ids': self.tag_input_ids[idx],
            'tag_attention_mask': self.tag_attention_masks[idx]
        }

In [ ]:
torch.cuda.is_available()

True

In [ ]:
descriptions = data['depicts']
tags = data['extracted_tags']


In [ ]:
train_descriptions, val_descriptions, train_tags, val_tags = train_test_split(descriptions, tags, test_size=0.2, random_state=42)


In [ ]:
train_dataset = TagsDataset(descriptions, tags)
val_dataset = TagsDataset(val_descriptions, val_tags)
train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE)


In [ ]:
model = BertForTagPrediction().to(Config.DEVICE)
optimizer = optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Training Loop
model.train()
for epoch in range(Config.EPOCHS):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(Config.DEVICE)
        attention_mask = batch['attention_mask'].to(Config.DEVICE)
        tag_input_ids = batch['tag_input_ids'].to(Config.DEVICE)
        tag_attention_mask = batch['tag_attention_mask'].to(Config.DEVICE)

        optimizer.zero_grad()
        similarities = model(input_ids, attention_mask, tag_input_ids, tag_attention_mask)
        # Simple example of maximizing similarity by minimizing negative similarity
        loss = -similarities.mean()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{Config.EPOCHS}, Average Loss: {avg_loss:.4f}')

Epoch 1/3, Average Loss: -0.9999
Epoch 2/3, Average Loss: -1.0000
Epoch 3/3, Average Loss: -1.0000


In [ ]:
torch.save(model.state_dict(), 'bert_tag_prediction_model.pth')
torch.save(optimizer.state_dict(), 'optimizer_state.pth')

# Testing

In [ ]:
# Load the model's state dictionary
model.load_state_dict(torch.load('bert_tag_prediction_model.pth'))

# If you also need to resume training, reinitialize the optimizer and load its state
optimizer = optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)
optimizer.load_state_dict(torch.load('optimizer_state.pth'))

In [ ]:
from sklearn.metrics import accuracy_score

# Switch to evaluation mode
model.eval()

# Initialize lists to store predictions and ground truth labels
all_predictions = []
all_ground_truths = []

# Iterate over the test data
for batch in val_loader:
    input_ids = batch['input_ids'].to(Config.DEVICE)
    attention_mask = batch['attention_mask'].to(Config.DEVICE)
    tag_input_ids = batch['tag_input_ids'].to(Config.DEVICE)
    tag_attention_mask = batch['tag_attention_mask'].to(Config.DEVICE)

    # Forward pass
    with torch.no_grad():
        similarities = model(input_ids, attention_mask, tag_input_ids, tag_attention_mask)

    # Convert similarities to binary predictions using a threshold
    threshold = 0.5
    binary_predictions = (similarities > threshold).int().tolist()
    all_predictions.extend(binary_predictions)

    # Assume all ground truth labels are positive (since we don't have ground truth labels)
    # You can remove this assumption if you have ground truth labels available
    all_ground_truths.extend([1] * len(binary_predictions))

# Calculate accuracy
accuracy = accuracy_score(all_ground_truths, all_predictions)
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 1.0000


In [ ]:
# Switch to evaluation mode
model.eval()

# Initialize a dictionary to store predictions
all_predictions = {}

# Iterate over the validation data
for batch in val_loader:
    input_ids = batch['input_ids'].to(Config.DEVICE)
    attention_mask = batch['attention_mask'].to(Config.DEVICE)
    tag_input_ids = batch['tag_input_ids'].to(Config.DEVICE)
    tag_attention_mask = batch['tag_attention_mask'].to(Config.DEVICE)

    # Forward pass
    with torch.no_grad():
        similarities = model(input_ids, attention_mask, tag_input_ids, tag_attention_mask)

    # Convert similarities to probabilities using sigmoid function
    probabilities = torch.sigmoid(similarities).tolist()

    # Extract the tag suggestions
    # Here, we use the batch index as the tag suggestion
    for i, probs in enumerate(probabilities):
        tag = f"tag_{i}"  # Create a tag name based on the batch index
        all_predictions[tag] = probs

# Print the dictionary of tag suggestions and their probabilities
print(all_predictions)

{'tag_0': 0.7310585379600525, 'tag_1': 0.7310585379600525, 'tag_2': 0.7310585379600525, 'tag_3': 0.7310585379600525, 'tag_4': 0.7310585379600525, 'tag_5': 0.7310585379600525, 'tag_6': 0.7310585975646973, 'tag_7': 0.7310585379600525}


# Q numbers

In [ ]:
def get_qnums(tag_list):
    qnum_dict = {}
    for s in tag_list:
        try:
            url = f'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={s}&languages=en&props=labels&format=json'
            response = requests.get(url).json()
            qnum = next(iter(response['entities']))
            if qnum.startswith('Q'):
                qnum_dict[s] = qnum
            else:
                #qnum_dict[s] = 'None'
                pass
        except:
            qnum_dict[s] = None
    # qnum_list = list(qnum_dict.values())
    # return qnum_list
    return qnum_dict

In [ ]:
test_qnums = get_qnums(test)

In [ ]:
test_qnums

{'2018 Summer Youth Olympics': 'Q870879',
 '2018': 'Q25291',
 'Buenos Aires': 'Q1486',
 'Yin Dehang': 'Q93888038'}

In [ ]:
q_num_list = list(test_qnums.values())
q_num_list

['Q870879', 'Q25291', 'Q1486', 'Q93888038']

In [ ]:
tag_suggestions = list(test_qnums.keys())
tag_suggestions

['2018 Summer Youth Olympics', '2018', 'Buenos Aires', 'Yin Dehang']